# 4주차 분석 실습 과제

파이썬 머신러닝 완벽가이드의 코드를 참고함.

## 1. 보팅(Voting) 실습

원하는 데이터셋을 불러와 예측 분석 수행하기.
예시에서는 breast cancer 데이터셋을 이용.

In [143]:
import pandas as pd
import numpy as np

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

* iris 데이터 사용

In [147]:
iris = datasets.load_iris()

In [148]:
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename']

In [149]:
data_df = pd.DataFrame(iris.data, columns=iris.feature_names)
data_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [198]:
X = iris.data[:, [0, 2]]
y = iris.target

보팅 분류기에는 다양한 개별 모델을 넣어서 사용할 수 있다.

여기서는 로지스틱 회귀, KNN, 의사결정나무 알고리즘을 제시했지만, 다른 모델을 추가하거나 변경 가능하다.

In [217]:
lr = LogisticRegression(max_iter=100, C=1, solver='liblinear')
svm = SVC(gamma ='auto', probability = True)
dtc = DecisionTreeClassifier(criterion="entropy")
knc = KNeighborsClassifier(n_neighbors=11)

base_methods=[('LogisticReg', lr), 
              ('SVM', svm), 
              ('DecisionTree',dtc),   
              ('KNeighbors',knc)]

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [219]:
vote_model=VotingClassifier(estimators=base_methods, 
       voting='hard',
       weights=[1,2,1,2])

In [220]:
for clf in (lr, svm, dtc, knc) :
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, format(accuracy_score(y_test, y_pred)))

LogisticRegression 0.9666666666666667
SVC 1.0
DecisionTreeClassifier 1.0
KNeighborsClassifier 1.0


In [221]:
# 개별 모델의 정확도
classifiers = [lr, dtc]

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

LogisticRegression 정확도: 0.9667
DecisionTreeClassifier 정확도: 1.0000


### 실습 결과

정확도가 가장 높았던 조합과 그 정확도를 작성해주세요!

**결과**

* 사용한 알고리즘 조합
> + svm = SVC(gamma ='auto', probability = True)
> + dtc = DecisionTreeClassifier(criterion="entropy")
> + knc = KNeighborsClassifier(n_neighbors=11)


* 정확도 :
> +  셋다 결과 1
> +  오버피팅


## 2. 랜덤포레스트(RandomForest) 실습

보팅 실습과 동일한 데이터셋 이용.

In [237]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes = 16, n_jobs = -1)

### 2.1 첫번째 실습

In [238]:
from sklearn.ensemble import RandomForestClassifier

In [239]:
data_df = pd.DataFrame(iris.data, columns=iris.feature_names)
data_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [240]:
X = iris.data[:, [0, 2]]
y = iris.target

In [241]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)

In [242]:
# 랜덤포레스트 학습, 예측, 정확도 평가
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [243]:
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜덤포레스트 정확도: {0:.4f}'.format(accuracy))

랜덤포레스트 정확도: 0.9333


### 2.2 GridSearchCV 이용한 두번째 실습

다양한 파라미터 값을 변화시키면서 최적의 하이퍼 파라미터 값을 찾고, 정확도를 높여 보세요.

* 추가, 변경 가능한 하이퍼 파라미터
> n_estimators, max_depth, max_features, min_samples_leaf, min_samples_split, max_leaf_nodes ......

In [244]:
from sklearn.model_selection import GridSearchCV

In [245]:
# 하이퍼 파라미터 변경해가며 정확도 높여보기.
model_params = {
    'n_estimators': [50, 150, 250],
    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
    'min_samples_split': [2, 4, 6]
}

In [246]:
# create random forest classifier model
rf_clf = RandomForestClassifier(random_state=1)

In [247]:
# set up grid search meta-estimator
grid_cv = GridSearchCV(rf_model, model_params, cv=5, n_jobs = -1)

In [248]:
# train the grid search meta-estimator to find the best model
grid_cv = clf.fit(X, y)

In [249]:
# print winning set of hyperparameters
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_features': 1.0, 'min_samples_split': 2, 'n_estimators': 50}
최고 예측 정확도: 0.9600


* 0.9333 에서 0.9600 으로 올렸따